In [159]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
import re
import string
# Any results you write to the current directory are saved as output.

from keras.models import Model
from keras.layers import Dense, Embedding, Input, LeakyReLU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, CuDNNLSTM, CuDNNGRU
from keras.layers import Dense, Embedding, Input, LeakyReLU, merge, Conv2D, Conv1D, PReLU,ELU,Concatenate, Convolution1D
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, GRU, Dropout, CuDNNGRU, Reshape, MaxPool2D,Flatten
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.backend.tensorflow_backend import set_session
from keras.optimizers import RMSprop
import os
import tensorflow as tf

In [2]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [4]:
pd.read_csv('../input/train.csv').head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
max_features = 20000
maxlen = 100

In [6]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [78]:
' '.join('a b c'.split())

'a b c'

In [80]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
train_text = pd.read_csv('../input/train.csv').comment_text.apply(lambda x: " ".join(regex.sub(' ', x).replace('\n', ' ').split()))
test_text = pd.read_csv('../input/test.csv').comment_text.apply(lambda x: " ".join(regex.sub(' ', x).replace('\n', ' ').split()) if isinstance(x, basestring) else 'Steeve')

In [81]:
train_text.head()

0    Explanation Why the edits made under my userna...
1    D aww He matches this background colour I m se...
2    Hey man I m really not trying to edit war It s...
3    More I can t make any real suggestions on impr...
4    You sir are my hero Any chance you remember wh...
Name: comment_text, dtype: object

In [7]:
list_sentences_train = train["comment_text"].fillna("Steeve").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("Steeve").values

In [92]:
list_sentences_train = train_text.values
list_sentences_test = test_text.values

In [93]:
list_sentences_train

array(['Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalisms just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now 89 205 38 27',
       'D aww He matches this background colour I m seemingly stuck with Thanks talk 21 51 January 11 2016 UTC',
       'Hey man I m really not trying to edit war It s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page He seems to care more about the formatting than the actual info',
       ...,
       'Spitzer Umm theres no actual article for prostitution ring Crunch Captain',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it',
       'And I really don t think you understand I came here and my idea was bad right away What kind of community goes you have bad ideas go away instead

In [94]:
list_sentences_test

array(['Yo bitch Ja Rule is more succesful then you ll ever be whats up with you and hating you sad mofuckas i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me Ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother too fuckin white boys get things right next time',
       'From RfC The title is fine as it is IMO',
       'Sources Zawe Ashton on Lapland \xe2\x80\x94', ...,
       'Okinotorishima categories I see your changes and agree this is more correct I had gotten confused but then found this while acknowledging Japan s territorial rights to Okinotorishima itself However is there a category for did not acknowledge Japan s claim to an exclusive economic zone EEZ stemming from Okinotorishima That is is there a category for disputed EEZ s',
       'One of the founding nations of the EU Germany has a Law of Return quite similar to Israel s This isn t actually true is it Germany allows 

In [95]:
tokenizer = text.Tokenizer(num_words=10)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)
word_index = tokenizer.word_index

In [96]:
import cPickle as pickle
with open('word_index_ft.p', 'wb') as fp:
    pickle.dump(word_index, fp)

In [165]:
embeddings_index={}
f = open( '../input/glove.twitter.27B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_dim = 200
embedding_matrix = np.zeros((max_features, embedding_dim))

In [97]:
import h5py
with h5py.File('toxic_token_ft.h5','w') as f:
    f.create_dataset("X_t", data = X_t )
    f.create_dataset("X_te", data = X_te )
    f.create_dataset("y", data = y )
    f.create_dataset("embedding_matrix", data=embedding_matrix)

In [169]:
def get_model():
    embed_size = embedding_dim
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.4)(x)
    x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
#     x = Dropout(0.4)(x)
#     x = Bidirectional(CuDNNLSTM(50, return_sequences=False))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.4)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.layers[1].set_weights([embedding_matrix])
    model.layers[1].trainable = False
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [161]:
def get_cnn_model():
    global embedding_matrix, embedding_dim
    filter_sizes = [3,8,5]
    num_filters = 10
    drop = 0.1
    
    embed_size = embedding_dim
    inputs = Input(shape=(maxlen, ), dtype='int32')
    embedding = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=maxlen)(inputs)
    reshape = Reshape((maxlen,embedding_dim,1))(embedding)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
    conv_0 = Dropout(0.1)(conv_0)
    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
    # maxpool_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(maxpool_0)
    # maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(maxpool_0)
    conv_1 = Dropout(0.1)(conv_1)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

    concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1])
    # concatenated_tensor = maxpool_0
    flatten = Flatten()(concatenated_tensor)
    flatten = Dense(30, activation='relu')(flatten)
    dropout = Dropout(drop)(flatten)
    output = Dense(6, activation='sigmoid')(dropout)
    model = Model(inputs=inputs, outputs=output)
#     model.layers[1].set_weights([embedding_matrix])
#     model.layers[1].trainable = False
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [148]:
from sklearn.metrics import roc_auc_score


import numpy as np


def _train_model(model, batch_size, train_x, train_y, val_x, val_y):
    best_auc = -1
    best_weights = None
    best_epoch = 0

    current_epoch = 0

    while True:
        model.fit(train_x, train_y, batch_size=batch_size, epochs=1)
        y_pred = model.predict(val_x, batch_size=batch_size)

        total_loss = 0
#         for j in range(6):
#             auc_score = auc(val_y[:, j], y_pred[:, j])
#             aucs += auc_score
        aucs = roc_auc_score(val_y, y_pred)

#         aucs /= 6.

        print("Epoch {0} loss {1} best_loss {2}".format(current_epoch, aucs, best_auc))

        current_epoch += 1
        if aucs > best_auc or best_auc == -1:
            best_auc = aucs
            best_weights = model.get_weights()
            best_epoch = current_epoch
        else:
            if current_epoch - best_epoch == 5:
                break

    model.set_weights(best_weights)
    return model

In [149]:
def train_folds(X, y, fold_count, batch_size, get_model_func):
    fold_size = len(X) // fold_count
    models = []
    for fold_id in range(0, fold_count):
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size

        if fold_id == fold_size - 1:
            fold_end = len(X)

        train_x = np.concatenate([X[:fold_start], X[fold_end:]])
        train_y = np.concatenate([y[:fold_start], y[fold_end:]])

        val_x = X[fold_start:fold_end]
        val_y = y[fold_start:fold_end]

        model = _train_model(get_model_func(), batch_size, train_x, train_y, val_x, val_y)
        models.append(model)

    return models

In [167]:
get_model_func = lambda: get_model()
get_cnn_model_func = lambda: get_cnn_model()
# model = get_model()
# model.load_weights('weights_base.pre_trained.hdf5')
fold_count=5
batch_size = 32
epochs = 5

In [170]:
models = train_folds(X_t, y, fold_count, batch_size, get_model_func)

Epoch 1/1
127657/127657 [==============================] - 81s 631us/step - loss: 0.3045 - acc: 0.9632
Epoch 0 loss 0.5 best_loss -1
Epoch 1/1
102144/127657 [=======================>......] - ETA: 15s - loss: 0.1522 - acc: 0.9631

KeyboardInterrupt: 

In [ ]:
fname='2bilstm300-50-50ft'
file_path="weights/"+fname+".hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=3)

In [111]:
callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 161s 1ms/step - loss: 0.2583 - acc: 0.9634 - val_loss: 0.1455 - val_acc: 0.9626
Epoch 2/5
143613/143613 [==============================] - 158s 1ms/step - loss: 0.1411 - acc: 0.9634 - val_loss: 0.1439 - val_acc: 0.9626
Epoch 3/5
143613/143613 [==============================] - 158s 1ms/step - loss: 0.1408 - acc: 0.9634 - val_loss: 0.1440 - val_acc: 0.9626
Epoch 4/5
 81184/143613 [===============>..............] - ETA: 1:05 - loss: 0.1403 - acc: 0.9636

KeyboardInterrupt: 

In [114]:
model.load_weights(file_path)

y_test = model.predict(X_t, verbose=1)

159571/159571 [==============================] - 69s 434us/step


In [115]:
y_test

array([[0.09585557, 0.00939463, 0.05202468, 0.00287224, 0.04876665,
        0.00813725],
       [0.09585557, 0.00939463, 0.05202468, 0.00287224, 0.04876665,
        0.00813725],
       [0.09585557, 0.00939463, 0.05202468, 0.00287224, 0.04876665,
        0.00813725],
       ...,
       [0.09585557, 0.00939463, 0.05202468, 0.00287224, 0.04876665,
        0.00813725],
       [0.09585557, 0.00939463, 0.05202468, 0.00287224, 0.04876665,
        0.00813725],
       [0.09585557, 0.00939463, 0.05202468, 0.00287224, 0.04876665,
        0.00813725]], dtype=float32)

In [107]:
sample_submission = pd.read_csv("../input/sample_submission.csv")

sample_submission[list_classes] = y_test



sample_submission.to_csv("output/"+fname+".csv", index=False)